# Tutorial on circuit creation

In this tutorial, we elaborate the method of building a circuit on Amazon Braket. Specifically, we discuss how to build (parametrized) circuits and display them graphically, how to append circuits to each other, how to get information from the circuit created, and how to find the gates supported by a certain device.

In [1]:
# Import the circuits class
from braket.circuits import Circuit

## Build a circuit

Let's get started to build a GHZ state with four qubits. By calling `Circuit()` we create an empty circuit, and we can just add gates to the circuit. 

In [2]:
# build a GHZ state with four qubits. Here 'cnot(control=0, target=1)' can be simplified as 'cnot(0,1)'
my_circuit = Circuit().h(0).cnot(control=0, target=1).cnot(control=1, target=2).cnot(control=2, target=3)

# visualize the circuit
print(my_circuit)

T  : |0|1|2|3|
              
q0 : -H-C-----
        |     
q1 : ---X-C---
          |   
q2 : -----X-C-
            | 
q3 : -------X-

T  : |0|1|2|3|


We can also build a circuit with parameterized gates.

In [3]:
# build a circuit with some parametrized gates 
my_circuit = Circuit().rx(0, 0.15).ry(1, 0.2).rz(2, 0.25).h([1,3]).cnot(0,2)
print(my_circuit)

T  : |   0    | 1 |
                   
q0 : -Rx(0.15)---C-
                 | 
q1 : -Ry(0.2)--H-|-
                 | 
q2 : -Rz(0.25)---X-
                   
q3 : -H------------

T  : |   0    | 1 |


Or build a circuit with self-defined unitary gates.

In [4]:
import numpy as np

# define and print the self-defined unitary
my_gate = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
print('The self-defined gate is:\n', my_gate)

# build a circuit with the self-defined gate U
my_circuit = Circuit().unitary(matrix=my_gate, targets=[0,1]).h(1).cnot(0,1)
print('\n',my_circuit)

The self-defined gate is:
 [[1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]]

 T  : |0|1|2|
            
q0 : -U---C-
      |   | 
q1 : -U-H-X-

T  : |0|1|2|


## Append gates to a circuit

We can append circuits to existing circuits with ```add_circuit()``` or `add()`. 

In [5]:
# append two circuits with add_circuit() functionality
circuit2 = Circuit().rz(0, 0.1)
my_circuit = my_circuit.add_circuit(circuit2)

# or with add()
my_circuit = my_circuit.add(circuit2)
print(my_circuit)

T  : |0|1|2|   3   |   4   |
                            
q0 : -U---C-Rz(0.1)-Rz(0.1)-
      |   |                 
q1 : -U-H-X-----------------

T  : |0|1|2|   3   |   4   |


We can specify the location of the circuit to be appended.

In [6]:
circuit2 = Circuit().cnot(0, 1)

# specify where to append the new piece of circuit
my_circuit = my_circuit.add_circuit(circuit2, target={0,2})
print(my_circuit)

# Swapping qubits when appending circuits can be easily down with the target_mapping
my_circuit = my_circuit.add_circuit(circuit2, target_mapping={0: 1, 1: 0})
print(my_circuit)

T  : |0|1|2|   3   |   4   |5|
                              
q0 : -U---C-Rz(0.1)-Rz(0.1)-C-
      |   |                 | 
q1 : -U-H-X-----------------|-
                            | 
q2 : -----------------------X-

T  : |0|1|2|   3   |   4   |5|
T  : |0|1|2|   3   |   4   |5|6|
                                
q0 : -U---C-Rz(0.1)-Rz(0.1)-C-X-
      |   |                 | | 
q1 : -U-H-X-----------------|-C-
                            |   
q2 : -----------------------X---

T  : |0|1|2|   3   |   4   |5|6|


We can also copy a circuit using `copy()`

In [7]:
my_2 = my_circuit.copy()

my_2.cnot(0,1)
print(my_2)

T  : |0|1|2|   3   |   4   |5|6|7|
                                  
q0 : -U---C-Rz(0.1)-Rz(0.1)-C-X-C-
      |   |                 | | | 
q1 : -U-H-X-----------------|-C-X-
                            |     
q2 : -----------------------X-----

T  : |0|1|2|   3   |   4   |5|6|7|


## Obtain information of the circuit created

Amazon Braket offers several functions to help do quick and simple inspections on a circuit.

As we can see from the displayed circuit, time is sliced up into moments. The circuit is rearranged such as to allow gates to be applied in parallel if they are at different qubits. 

The depth of a circuit is the total number of moments.

In [8]:
# get circuit depth
circuit_depth = my_circuit.depth
print(my_circuit)
print()
print('Total circuit depth:', circuit_depth)

# get qubit number
qubit_count = my_circuit.qubit_count
print('Total qubit number:', qubit_count)
print()

# show moments of the circuit
my_moments = my_circuit.moments
for moment in my_moments:
    print(moment)

T  : |0|1|2|   3   |   4   |5|6|
                                
q0 : -U---C-Rz(0.1)-Rz(0.1)-C-X-
      |   |                 | | 
q1 : -U-H-X-----------------|-C-
                            |   
q2 : -----------------------X---

T  : |0|1|2|   3   |   4   |5|6|

Total circuit depth: 7
Total qubit number: 3

MomentsKey(time=0, qubits=QubitSet([Qubit(0), Qubit(1)]))
MomentsKey(time=1, qubits=QubitSet([Qubit(1)]))
MomentsKey(time=2, qubits=QubitSet([Qubit(0), Qubit(1)]))
MomentsKey(time=3, qubits=QubitSet([Qubit(0)]))
MomentsKey(time=4, qubits=QubitSet([Qubit(0)]))
MomentsKey(time=5, qubits=QubitSet([Qubit(0), Qubit(2)]))
MomentsKey(time=6, qubits=QubitSet([Qubit(1), Qubit(0)]))


## Supported gate sets by different devices

If you need to submit a circuit to run on a certain device, please keep in mind that different devices provided by Amazon Braket support different gate sets. The local simulator and the cloud-based simulator support the max number of gates, while the IonQ and Rigetti devices each support a subset of the whole gate set.

In [9]:
from braket.aws import AwsDevice
from braket.devices import LocalSimulator

# the local simulator
device = LocalSimulator()
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gates supported by the local simulator include:\n', supported_gates)
print('\n')

# the cloud-based simulator
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gates supported by the cloud-based simulator include:\n', supported_gates)
print('\n')

# the Rigetti device
device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-8")
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gates supported by the Rigetti device include:\n', supported_gates)
print('\n') 

# the IonQ device
device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
supported_gates = device.properties.action['braket.ir.jaqcd.program'].supportedOperations
# print the supported gate set
print('Gates supported by the IonQ device include:\n', supported_gates)

Gates supported by the local simulator include:
 ['CCNot', 'CNot', 'CPhaseShift', 'CPhaseShift00', 'CPhaseShift01', 'CPhaseShift10', 'CSwap', 'CY', 'CZ', 'H', 'I', 'ISwap', 'PSwap', 'PhaseShift', 'Rx', 'Ry', 'Rz', 'S', 'Si', 'Swap', 'T', 'Ti', 'Unitary', 'V', 'Vi', 'X', 'XX', 'XY', 'Y', 'YY', 'Z', 'ZZ']


Gates supported by the cloud-based simulator include:
 ['ccnot', 'cnot', 'cphaseshift', 'cphaseshift00', 'cphaseshift01', 'cphaseshift10', 'cswap', 'cy', 'cz', 'h', 'i', 'iswap', 'pswap', 'phaseshift', 'rx', 'ry', 'rz', 's', 'si', 'swap', 't', 'ti', 'unitary', 'v', 'vi', 'x', 'xx', 'xy', 'y', 'yy', 'z', 'zz']


Gates supported by the Rigetti device include:
 ['cz', 'xy', 'ccnot', 'cnot', 'cphaseshift', 'cphaseshift00', 'cphaseshift01', 'cphaseshift10', 'cswap', 'h', 'i', 'iswap', 'phaseshift', 'pswap', 'rx', 'ry', 'rz', 's', 'si', 'swap', 't', 'ti', 'x', 'y', 'z']


Gates supported by the IonQ device include:
 ['x', 'y', 'z', 'rx', 'ry', 'rz', 'h', 'cnot', 's', 'si', 't', 'ti', 'v', '

## Submit the circuit to the local simulator and obtain the results

Here we submit our created circuit to the local simulator and obtain the results.

In [10]:
# set up device
device = LocalSimulator()

# run circuit
result = device.run(my_circuit, shots=1000).result()
# get measurement shots
counts = result.measurement_counts
# print counts
print(counts)

Counter({'110': 501, '000': 499})
